In [11]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import torch
import torch.nn.functional as F
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from torch.autograd import Variable
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

df = pd.read_csv('/content/drive/MyDrive/deep_learning/Final_Project/data.csv')

df.head()

,name,experimental_band_gap,phonon_cutoff_frequency,mean_phonon_frequency,electronic_contribution_of_dielectric_constant,total_dielectric_constant,nearest_neighbor_distance,density,bulk_modulus,structure,category,log_breakdown_field
0,LiF,13.6,19.599,10.435,2.068,7.936,1.943,2.938,86.582,RS,Alkali_metal_halides,8.482519
1,LiCl,9.4,12.531,6.301,3.150,10.003,2.477,2.317,40.537,RS,Alkali_metal_halides,6.521799
2,LiBr,7.6,11.188,4.934,3.664,11.287,2.653,3.862,32.869,RS,Alkali_metal_halides,6.171909
3,LiI,6.1,9.701,4.113,4.465,12.667,2.897,4.569,25.476,RS,Alkali_metal_halides,5.944111
4,NaF,11.5,12.275,7.416,1.821,4.170,2.227,3.156,61.943,RS,Alkali_metal_halides,6.968568


In [12]:
from sklearn.preprocessing import LabelEncoder
import numpy as np

## categoty의 범주형 Data를 숫자형 Data로 변환해주어야 함
encoder = LabelEncoder()
df['structure'] = encoder.fit_transform(df['structure'])
df['category'] = encoder.fit_transform(df['category'])

df.head()

,name,experimental_band_gap,phonon_cutoff_frequency,mean_phonon_frequency,electronic_contribution_of_dielectric_constant,total_dielectric_constant,nearest_neighbor_distance,density,bulk_modulus,structure,category,log_breakdown_field
0,LiF,13.6,19.599,10.435,2.068,7.936,1.943,2.938,86.582,1,0,8.482519
1,LiCl,9.4,12.531,6.301,3.150,10.003,2.477,2.317,40.537,1,0,6.521799
2,LiBr,7.6,11.188,4.934,3.664,11.287,2.653,3.862,32.869,1,0,6.171909
3,LiI,6.1,9.701,4.113,4.465,12.667,2.897,4.569,25.476,1,0,5.944111
4,NaF,11.5,12.275,7.416,1.821,4.170,2.227,3.156,61.943,1,0,6.968568


In [32]:
# 특징 및 타겟 정의
X = df.drop(['name', 'log_breakdown_field', 'structure', 'category'], axis=1).values
y = df[['log_breakdown_field', 'category']].values

print(f"Unique target values: {np.unique(y[:, 0])}")
# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y[:, 0], test_size=0.3)

# 데이터 전처리
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

# X_train = Variable(torch.from_numpy(X_train)).float()
# y_train = torch.LongTensor(y_train)
# X_test  = Variable(torch.from_numpy(X_test)).float()
# y_test  = torch.LongTensor(y_test)

# PyTorch 텐서로 변환
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.long)

# 타겟 값 범위 재확인
print(f"Train target values: {y_train.unique()}")
print(f"Test target values: {y_test.unique()}")

Unique target values: [2.360854   2.72785283 3.0056826  3.5807373  3.74478709 3.76815264
 3.81330703 3.84801768 3.910021   3.96271612 4.34250588 4.35926965
 4.43319492 4.5196123  4.52396013 4.57057874 4.62104354 4.62399194
 4.69043003 4.69957086 4.7022969  4.73532087 4.74666975 4.75013596
 4.76558691 4.76983681 4.81705055 4.82991246 4.89185176 4.95017733
 4.95512258 4.96703166 4.98086314 5.01196774 5.03630304 5.04277975
 5.17727904 5.1885025  5.20180647 5.20345709 5.23590991 5.23750523
 5.2585365  5.26579417 5.29179615 5.33513134 5.34185567 5.34472374
 5.354225   5.44328314 5.44457987 5.47185042 5.49100171 5.49264984
 5.54204755 5.61495126 5.63085329 5.66608061 5.67641128 5.7905716
 5.84701664 5.85277703 5.86505062 5.94411085 5.97761914 6.03906353
 6.06099061 6.17190934 6.17336935 6.23988591 6.24900953 6.28245365
 6.41689567 6.52179864 6.53915234 6.78185186 6.96856812 7.38392734
 7.63148016 7.87245515 8.48251893]
Train target values: tensor([2, 3, 4, 5, 6, 7, 8])
Test target values: te

In [36]:
output_dim = max(np.unique(y_train)) + 1 # 타겟 클래스 수에 맞게 조정
print(output_dim)

9


In [37]:
# 커스텀 데이터셋 클래스 정의
class CustomDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# DataLoader 생성
train_dataset = CustomDataset(X_train, y_train)
test_dataset = CustomDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# 신경망 정의
class FeedForwardNN(nn.Module):
    def __init__(self, input_dim, hidden, out):
        super(FeedForwardNN, self).__init__()
        self.layer1 = nn.Linear(input_dim, hidden)
        self.layer2 = nn.Linear(hidden, hidden)
        self.layer3 = nn.Linear(hidden, out)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.layer1(x))
        x = self.relu(self.layer2(x))
        x = self.layer3(x)  # 출력층에는 활성화 함수가 필요 없음
        return x

# 모델, 옵티마이저, 손실 함수 초기화
input_dim = X_train.shape[1]
hidden_dim = 25
output_dim = max(np.unique(y_train)) + 1   # 타겟 클래스 수에 맞게 조정
ffn = FeedForwardNN(input_dim, hidden_dim, output_dim)
optimizer = torch.optim.Adam(ffn.parameters(), lr=0.001)
loss_fn = nn.CrossEntropyLoss()

# 학습 루프
epochs = 1000
train_losses = np.zeros(epochs)
test_losses = np.zeros(epochs)

for epoch in range(epochs):
    ffn.train()
    epoch_train_loss = 0
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        y_pred = ffn(X_batch)
        loss_train = loss_fn(y_pred, y_batch)
        loss_train.backward()
        optimizer.step()
        epoch_train_loss += loss_train.item()

    epoch_train_loss /= len(train_loader)

    ffn.eval()
    epoch_test_loss = 0
    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            y_pred_test = ffn(X_batch)
            loss_test = loss_fn(y_pred_test, y_batch)
            epoch_test_loss += loss_test.item()

    epoch_test_loss /= len(test_loader)

    train_losses[epoch] = epoch_train_loss
    test_losses[epoch] = epoch_test_loss

    if (epoch + 1) % 25 == 0:
        print(f"Epoch {epoch+1}/{epochs}, Train Loss: {epoch_train_loss:.3f}, Test Loss: {epoch_test_loss:.3f}")


Epoch 25/1000, Train Loss: 1.832, Test Loss: 1.903
Epoch 50/1000, Train Loss: 1.283, Test Loss: 1.503
Epoch 75/1000, Train Loss: 0.929, Test Loss: 1.188
Epoch 100/1000, Train Loss: 0.688, Test Loss: 1.095
Epoch 125/1000, Train Loss: 0.560, Test Loss: 1.116
Epoch 150/1000, Train Loss: 0.470, Test Loss: 1.162
Epoch 175/1000, Train Loss: 0.411, Test Loss: 1.224
Epoch 200/1000, Train Loss: 0.361, Test Loss: 1.294
Epoch 225/1000, Train Loss: 0.309, Test Loss: 1.404
Epoch 250/1000, Train Loss: 0.275, Test Loss: 1.513
Epoch 275/1000, Train Loss: 0.243, Test Loss: 1.622
Epoch 300/1000, Train Loss: 0.219, Test Loss: 1.736
Epoch 325/1000, Train Loss: 0.187, Test Loss: 1.860
Epoch 350/1000, Train Loss: 0.176, Test Loss: 1.971
Epoch 375/1000, Train Loss: 0.152, Test Loss: 2.062
Epoch 400/1000, Train Loss: 0.134, Test Loss: 2.183
Epoch 425/1000, Train Loss: 0.134, Test Loss: 2.283
Epoch 450/1000, Train Loss: 0.122, Test Loss: 2.382
Epoch 475/1000, Train Loss: 0.114, Test Loss: 2.466
Epoch 500/1000,

In [15]:
class FeedForwardNN(nn.Module):
    def __init__(self, input_dim,hidden,out):
        super(FeedForwardNN,self).__init__()
        self.relu=nn.ReLU()
        self.layer1 = nn.Linear(input_dim, hidden)
        self.layer2 = nn.Linear(hidden,hidden)
        self.layer3 = nn.Linear(hidden,out)

    def forward(self,x):
        x = self.relu(self.layer1(x))
        x = self.relu(self.layer2(x))
        x= self.relu(self.layer3(x))
        return x

In [19]:
# 커스텀 데이터셋 클래스 정의
class CustomDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# DataLoader 생성
train_dataset = CustomDataset(X_train, y_train)
test_dataset = CustomDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [25]:
# 신경망 정의
class FeedForwardNN(nn.Module):
    def __init__(self, input_dim, hidden, out):
        super(FeedForwardNN, self).__init__()
        self.layer1 = nn.Linear(input_dim, hidden)
        self.layer2 = nn.Linear(hidden, hidden)
        self.layer3 = nn.Linear(hidden, out)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.layer1(x))
        x = self.relu(self.layer2(x))
        x = self.layer3(x)  # 출력층에는 활성화 함수가 필요 없음
        return x

# 모델, 옵티마이저, 손실 함수 초기화
input_dim = X_train.shape[1]
hidden_dim = 25
output_dim = len(np.unique(y_train))  # 타겟 클래스 수에 맞게 조정
ffn = FeedForwardNN(input_dim, hidden_dim, output_dim)
optimizer = torch.optim.Adam(ffn.parameters(), lr=0.001)
loss_fn = nn.CrossEntropyLoss()
ffn

FeedForwardNN(
  (layer1): Linear(in_features=8, out_features=25, bias=True)
  (layer2): Linear(in_features=25, out_features=25, bias=True)
  (layer3): Linear(in_features=25, out_features=7, bias=True)
  (relu): ReLU()
)

In [27]:
print(f"Unique target values: {np.unique(y[:, 0])}")
print(output_dim)

Unique target values: [2.360854   2.72785283 3.0056826  3.5807373  3.74478709 3.76815264
 3.81330703 3.84801768 3.910021   3.96271612 4.34250588 4.35926965
 4.43319492 4.5196123  4.52396013 4.57057874 4.62104354 4.62399194
 4.69043003 4.69957086 4.7022969  4.73532087 4.74666975 4.75013596
 4.76558691 4.76983681 4.81705055 4.82991246 4.89185176 4.95017733
 4.95512258 4.96703166 4.98086314 5.01196774 5.03630304 5.04277975
 5.17727904 5.1885025  5.20180647 5.20345709 5.23590991 5.23750523
 5.2585365  5.26579417 5.29179615 5.33513134 5.34185567 5.34472374
 5.354225   5.44328314 5.44457987 5.47185042 5.49100171 5.49264984
 5.54204755 5.61495126 5.63085329 5.66608061 5.67641128 5.7905716
 5.84701664 5.85277703 5.86505062 5.94411085 5.97761914 6.03906353
 6.06099061 6.17190934 6.17336935 6.23988591 6.24900953 6.28245365
 6.41689567 6.52179864 6.53915234 6.78185186 6.96856812 7.38392734
 7.63148016 7.87245515 8.48251893]
7


In [24]:
# 학습 루프
epochs = 1000
train_losses = np.zeros(epochs)
test_losses = np.zeros(epochs)

for epoch in range(epochs):
    ffn.train()
    epoch_train_loss = 0
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        y_pred = ffn(X_batch)
        loss_train = loss_fn(y_pred, y_batch)
        loss_train.backward()
        optimizer.step()
        epoch_train_loss += loss_train.item()

    epoch_train_loss /= len(train_loader)

    ffn.eval()
    epoch_test_loss = 0
    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            y_pred_test = ffn(X_batch)
            loss_test = loss_fn(y_pred_test, y_batch)
            epoch_test_loss += loss_test.item()

    epoch_test_loss /= len(test_loader)

    train_losses[epoch] = epoch_train_loss
    test_losses[epoch] = epoch_test_loss

    if (epoch + 1) % 25 == 0:
        print(f"Epoch {epoch+1}/{epochs}, Train Loss: {epoch_train_loss:.3f}, Test Loss: {epoch_test_loss:.3f}")

# 손실 값 시각화
plt.plot(train_losses, label='Train Loss')
plt.plot(test_losses, label='Test Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

IndexError: Target 7 is out of bounds.

모델 훈련

In [21]:
ffn = FeedForwardNN(input_dim = X_train.shape[1],hidden = 25,out=3)
optimizer = torch.optim.Adam(ffn.parameters(), lr=0.001)
loss_fn   = nn.CrossEntropyLoss()
ffn

FeedForwardNN(
  (layer1): Linear(in_features=8, out_features=25, bias=True)
  (layer2): Linear(in_features=25, out_features=25, bias=True)
  (layer3): Linear(in_features=25, out_features=3, bias=True)
  (relu): ReLU()
)

In [22]:
epochs = 1000
train_losses =np.zeros(epochs)
test_losses = np.zeros(epochs)

for epoch in range(epochs):
    y_pred = ffn(X_train)
    loss_train = loss_fn(y_pred,y_train)
    optimizer.zero_grad()
    loss_train.backward()
    optimizer.step()

    y_pred_test = ffn(X_test)
    loss_test = loss_fn(y_pred_test,y_test)
    train_losses[epoch] = loss_train.item()
    test_losses[epoch] = loss_test.item()


    if (epoch + 1) % 25 == 0:
            print(f"Epoch {epoch+1}/{epochs}, Train Loss: {loss_train.item():.3f}, Test Loss: {loss_test.item():.3f}")

IndexError: Target 5 is out of bounds.

In [18]:
print(y_train)
print(df['category'].unique())

tensor([5, 3, 5, 5, 4, 5, 3, 5, 4, 4, 4, 5, 4, 4, 7, 7, 6, 5, 5, 6, 5, 4, 5, 5,
        8, 6, 6, 3, 4, 3, 6, 4, 4, 5, 5, 5, 2, 5, 4, 4, 4, 5, 4, 5, 3, 5, 5, 5,
        5, 3, 5, 5, 6, 3, 4, 5, 4])
[0 6 5 1 7 4 3 2]


In [38]:
from sklearn.metrics import r2_score

model.eval()
predictions = []
actuals = []

with torch.no_grad():
    for inputs, targets in test_loader:
        outputs = model(inputs).squeeze()
        predictions.append(outputs.numpy())
        actuals.append(targets.numpy())

predictions = np.concatenate(predictions)
actuals = np.concatenate(actuals)

# 성능 평가
# mse = np.mean((predictions - actuals) ** 2)
# print(f'Mean Squared Error: {mse:.4f}')

r2 = r2_score(actuals, predictions)
print(f'R^2 Score: {r2:.4f}')

NameError: name 'model' is not defined